<a href="https://colab.research.google.com/github/Ashonet/S-P100_ANALYSIS/blob/main/SP100_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_intrinsic_value_dividend(symbol):
    try:
        data = yf.Ticker(symbol)
        dividends = data.dividends

        if len(dividends) < 2:
            return "Insufficient data for analysis", None, None  # Return both message and None value if insufficient data

        growth_rate = (dividends.iloc[-1] - dividends.iloc[0]) / dividends.iloc[0]  # Calculate growth rate based on dividends

        latest_dividend = dividends.iloc[-1]
        discount_rate = 0.1  # Set your required rate of return

        # Ensure growth rate is not greater than the discount rate to avoid issues
        growth_rate = min(growth_rate, discount_rate * 0.9)  # Set a maximum growth rate

        intrinsic_value = latest_dividend / (discount_rate - growth_rate)
        current_price = data.history(period="1d")['Close'][0]

        if current_price < intrinsic_value:
            return "Buy", intrinsic_value, "Price to Buy"
        elif current_price > intrinsic_value:
            return "Sell", intrinsic_value, "Price to Sell"
        else:
            # For 'Hold', define a price range as a percentage above and below the intrinsic value
            hold_range_percentage = 5  # Define the percentage for the hold range

            lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))  # Ensure positive lower bound
            upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None

def get_intrinsic_value_relative(symbol):
    try:
        data = yf.Ticker(symbol)
        earnings_per_share = data.info.get('trailingEps', None)
        book_value_per_share = data.info.get('bookValue', None)

        if earnings_per_share is None or np.isnan(earnings_per_share) or book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None, None
        # Relative Value
        intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)
        current_price = data.history(period="1d")['Close'][-1]

        if np.isnan(intrinsic_value):
            return "Unable to calculate intrinsic value", None, None

        hold_range_percentage = 5
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None

def calculate_residual_income(book_value, cost_of_equity, future_residual_income):
    # Calculate intrinsic value using the residual income model
    intrinsic_value = book_value * (1 + (future_residual_income * 0.01) / (1 + cost_of_equity * 0.01))
    return intrinsic_value

def estimate_future_residual_income(symbol):
    try:
        data = yf.Ticker(symbol)
        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None

        # Fetch historical data (assuming quarterly data)
        historical_data = data.history(period="1y")  # Adjust period as needed

        if historical_data.empty or len(historical_data) < 4:  # At least 4 quarters needed for trend estimation
            return "Insufficient historical data for analysis", None

        # Calculate past residual incomes based on historical book values
        historical_book_values = historical_data['Close'].to_list()  # Assuming 'Close' prices represent book values
        past_residual_incomes = [0]  # Initialize with zero as the starting residual income

        for i in range(1, len(historical_book_values)):
            residual_income = max(0, historical_book_values[i] - book_value_per_share)
            past_residual_incomes.append(residual_income)

        # Estimate growth rate based on past performance (simple average growth rate)
        average_growth_rate = sum(past_residual_incomes[-4:]) / len(past_residual_incomes[-4:])

        # Project future residual income using the average growth rate
        future_residual_income = past_residual_incomes[-1] + average_growth_rate

        return future_residual_income
    except Exception as e:
        print(f"Error estimating future residual income for {symbol}: {e}")
        return None

def get_beta(symbol):
    try:
        stock_data = yf.Ticker(symbol)
        stock_history = stock_data.history(period='max')

        market_index = '^GSPC'  # S&P 500 index
        market_data = yf.Ticker(market_index).history(period='max')

        cov_stock_market = stock_history['Close'].pct_change().cov(market_data['Close'].pct_change())
        var_market = market_data['Close'].pct_change().var()

        beta = cov_stock_market / var_market
        return beta
    except Exception as e:
        print(f"Error fetching beta for {symbol}: {e}")
        return None

def calculate_cost_of_equity(symbol):
    try:
        risk_free_rate = 0.04  # Replace with actual risk-free rate (e.g., 10-year Treasury yield)
        market_return = 0.08  # Replace with expected market return

        beta = get_beta(symbol)
        if beta is None:
            return None

        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
        return cost_of_equity
    except Exception as e:
        print(f"Error calculating cost of equity for {symbol}: {e}")
        return None

def get_intrinsic_value_residual_income(symbol, cost_of_equity):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return "Insufficient data for analysis", None, None

        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None, None

        # Calculate future residual income based on historical performance
        future_residual_income = estimate_future_residual_income(symbol)

        if future_residual_income is None:
            return "Unable to estimate future residual income", None, None

        # Call the function to calculate intrinsic value using estimated future residual income
        intrinsic_value = calculate_residual_income(book_value_per_share, cost_of_equity, future_residual_income)
        current_price = historical_data['Close'][-1]

        if np.isnan(intrinsic_value):
            return "Unable to calculate intrinsic value", None, None

        hold_range_percentage = 7
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None, None, None

def get_recommendation_for_symbol(symbol, valuation_method):
    # Placeholder logic to generate recommendations based on the valuation method
    # Replace this with the actual logic for obtaining recommendations for each method
    if valuation_method == 'dividend':
        # Example placeholder for dividend valuation method
        return np.random.choice(['Buy', 'Sell', 'Hold'])
    elif valuation_method == 'relative':
        # Example placeholder for relative valuation method
        return np.random.choice(['Buy', 'Sell', 'Hold'])
    elif valuation_method == 'residual_income':
        # Example placeholder for residual income valuation method
        return np.random.choice(['Buy', 'Sell', 'Hold'])
    else:
        return None

def analyze_stocks(stock_symbols, start_date, end_date, valuation_method):
    recommendations = {}
    for symbol in stock_symbols:
        stock_data = get_stock_data(symbol, start_date, end_date)
        if not stock_data.empty:
            try:
                # Get the recommendation for each symbol based on the specified valuation method
                recommendation = get_recommendation_for_symbol(symbol, valuation_method)
                recommendations[symbol] = recommendation
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")
                recommendations[symbol] = "Error in analysis"
        else:
            recommendations[symbol] = "No data available"
    return recommendations

# Get S&P 100 company symbols
sp100_symbols = get_sp100_companies()

# Example Date:
start_date = '2019-12-01'
end_date = '2022-12-20'

# Generate recommendations using different valuation methods for S&P 100 stocks
stock_recommendations_dividend = analyze_stocks(sp100_symbols, start_date, end_date, valuation_method='dividend')
stock_recommendations_relative = analyze_stocks(sp100_symbols, start_date, end_date, valuation_method='relative')
stock_recommendations_residual_income = analyze_stocks(sp100_symbols, start_date, end_date, valuation_method='residual_income')

# Define a function to calculate average recommendations across different methods
def average_recommendations(recommendations_list):
    average_dict = {}
    for symbol in recommendations_list[0].keys():
        values = [recommendations[symbol] for recommendations in recommendations_list if symbol in recommendations]
        if values:
            # Compute the average recommendation
            average_value = np.random.choice(values)  # Random choice for demonstration
            average_dict[symbol] = average_value
        else:
            average_dict[symbol] = "No recommendations available"
    return average_dict

# Combine recommendations for averaging
recommendations_list = [stock_recommendations_dividend, stock_recommendations_relative, stock_recommendations_residual_income]

# Calculate average recommendations
average_recommendations = average_recommendations(recommendations_list)

# Output the averaged recommendations
print("-------- Averaged Recommendations --------")
for symbol, recommendation in average_recommendations.items():
    print(f"{symbol}: {recommendation}")
print("-----------------------------------------")


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr

def get_stock_data(symbol, start_date, end_date):
    try:
        stock = yf.download(symbol, start=start_date, end=end_date)
        return stock
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame if there's an error

def get_sp100_companies():
    sp100_symbols = pd.read_html('https://en.wikipedia.org/wiki/S%26P_100')[2]
    sp100_symbols = sp100_symbols['Symbol'].tolist()
    return sp100_symbols

def get_intrinsic_value_dividend(symbol):
    try:
        data = yf.Ticker(symbol)
        dividends = data.dividends

        if len(dividends) < 2:
            return None  # Return only the intrinsic value if there's insufficient data

        growth_rate = (dividends.iloc[-1] - dividends.iloc[0]) / dividends.iloc[0]  # Calculate growth rate based on dividends

        latest_dividend = dividends.iloc[-1]
        discount_rate = 0.1  # Set your required rate of return

        # Ensure growth rate is not greater than the discount rate to avoid issues
        growth_rate = min(growth_rate, discount_rate * 0.9)  # Set a maximum growth rate

        intrinsic_value = latest_dividend / (discount_rate - growth_rate)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None

def get_intrinsic_value_relative(symbol):
    try:
        data = yf.Ticker(symbol)
        earnings_per_share = data.info.get('trailingEps', None)
        book_value_per_share = data.info.get('bookValue', None)

        if earnings_per_share is None or np.isnan(earnings_per_share) or book_value_per_share is None or np.isnan(book_value_per_share):
            return None  # Return None if financial data is insufficient or invalid

        # Relative Value
        intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None

def calculate_residual_income(book_value, cost_of_equity, future_residual_income):
    # Calculate intrinsic value using the residual income model
    intrinsic_value = book_value * (1 + (future_residual_income * 0.01) / (1 + cost_of_equity * 0.01))
    return intrinsic_value

def estimate_future_residual_income(symbol):
    try:
        data = yf.Ticker(symbol)
        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return "Insufficient or invalid financial data for analysis", None

        # Fetch historical data (assuming quarterly data)
        historical_data = data.history(period="1y")  # Adjust period as needed

        if historical_data.empty or len(historical_data) < 4:  # At least 4 quarters needed for trend estimation
            return "Insufficient historical data for analysis", None

        # Calculate past residual incomes based on historical book values
        historical_book_values = historical_data['Close'].to_list()  # Assuming 'Close' prices represent book values
        past_residual_incomes = [0]  # Initialize with zero as the starting residual income

        for i in range(1, len(historical_book_values)):
            residual_income = max(0, historical_book_values[i] - book_value_per_share)
            past_residual_incomes.append(residual_income)

        # Estimate growth rate based on past performance (simple average growth rate)
        average_growth_rate = sum(past_residual_incomes[-4:]) / len(past_residual_incomes[-4:])

        # Project future residual income using the average growth rate
        future_residual_income = past_residual_incomes[-1] + average_growth_rate

        return future_residual_income
    except Exception as e:
        print(f"Error estimating future residual income for {symbol}: {e}")
        return None

def get_beta(symbol):
    try:
        stock_data = yf.Ticker(symbol)
        stock_history = stock_data.history(period='max')

        market_index = '^GSPC'  # S&P 500 index
        market_data = yf.Ticker(market_index).history(period='max')

        cov_stock_market = stock_history['Close'].pct_change().cov(market_data['Close'].pct_change())
        var_market = market_data['Close'].pct_change().var()

        beta = cov_stock_market / var_market
        return beta
    except Exception as e:
        print(f"Error fetching beta for {symbol}: {e}")
        return None

def calculate_cost_of_equity(symbol):
    try:
        risk_free_rate = 0.04  # Replace with actual risk-free rate (e.g., 10-year Treasury yield)
        market_return = 0.08  # Replace with expected market return

        beta = get_beta(symbol)
        if beta is None:
            return None

        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
        return cost_of_equity
    except Exception as e:
        print(f"Error calculating cost of equity for {symbol}: {e}")
        return None

def get_intrinsic_value_residual_income(symbol, cost_of_equity):
    try:
        data = yf.Ticker(symbol)
        historical_data = data.history(period="max")

        if historical_data.empty:
            return None  # Return None if there's insufficient data for analysis

        book_value_per_share = data.info.get('bookValue', None)

        if book_value_per_share is None or np.isnan(book_value_per_share):
            return None  # Return None if financial data is insufficient or invalid for analysis

        # Calculate future residual income based on historical performance
        future_residual_income = estimate_future_residual_income(symbol)

        if future_residual_income is None:
            return None  # Return None if unable to estimate future residual income

        # Call the function to calculate intrinsic value using estimated future residual income
        intrinsic_value = calculate_residual_income(book_value_per_share, cost_of_equity, future_residual_income)
        return intrinsic_value

    except Exception as e:
        print(f"Error calculating intrinsic value for {symbol}: {e}")
        return None


def calculate_average_intrinsic_value(symbol):
    intrinsic_values = []

    # Get intrinsic value from dividend approach
    intrinsic_value_dividend = get_intrinsic_value_dividend(symbol)
    if intrinsic_value_dividend is not None:
        intrinsic_values.append(intrinsic_value_dividend)

    # Get intrinsic value from relative approach
    intrinsic_value_relative = get_intrinsic_value_relative(symbol)
    if intrinsic_value_relative is not None:
        intrinsic_values.append(intrinsic_value_relative)

    # Get intrinsic value from residual income approach
    cost_of_equity = calculate_cost_of_equity(symbol)
    if cost_of_equity is not None:
        intrinsic_value_residual_income = get_intrinsic_value_residual_income(symbol, cost_of_equity)
        if intrinsic_value_residual_income is not None:
            intrinsic_values.append(intrinsic_value_residual_income)

    if len(intrinsic_values) > 0:
        average_intrinsic_value = np.mean(intrinsic_values)
        return average_intrinsic_value
    else:
        return None

def get_stock_rating(symbol, current_price):
    hold_range_percentage = 5
    intrinsic_value = calculate_average_intrinsic_value(symbol)

    if intrinsic_value is not None:
        lower_hold_price = max(0, intrinsic_value * (1 - hold_range_percentage / 100))
        upper_hold_price = intrinsic_value * (1 + hold_range_percentage / 100)

        if lower_hold_price <= current_price <= upper_hold_price:
            return "Hold", (lower_hold_price, upper_hold_price), "Price Range to Hold"
        elif current_price < lower_hold_price:
            return "Buy", intrinsic_value, "Upper Price Target"
        else:
            return "Sell", intrinsic_value, "Lower Price Target"
    else:
        return None, None, None  # Return None if intrinsic value cannot be calculated

def main():
    # Get S&P 100 company symbols
    sp100_symbols = get_sp100_companies()

    # Example usage:
    start_date = '2019-12-01'
    end_date = '2023-12-20'

    for symbol in sp100_symbols:
        # Get historical data for the given symbol
        historical_data = get_stock_data(symbol, start_date, end_date)

        if historical_data.empty:
            print(f"Insufficient data for analysis for {symbol}")
        else:
            # Extracting the current price from historical data
            current_price = historical_data['Close'][-1]

            recommendation, intrinsic_value, price_type = get_stock_rating(symbol, current_price)

            if recommendation is not None and intrinsic_value is not None and price_type is not None:
                if recommendation == "Buy":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value:.2f}")
                elif recommendation == "Sell":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value:.2f}")
                elif recommendation == "Hold":
                    print(f"{symbol}: {recommendation} - {price_type} - {intrinsic_value[0]:.2f} to {intrinsic_value[1]:.2f}")
            else:
                print(f"No data available for {symbol}")

# Call the main function to execute the analysis
if __name__ == "__main__":
    main()


[*********************100%%**********************]  1 of 1 completed
AAPL: Sell - Lower Price Target - 21.90
[*********************100%%**********************]  1 of 1 completed
ABBV: Sell - Lower Price Target - 66.58
[*********************100%%**********************]  1 of 1 completed
ABT: Sell - Lower Price Target - 49.59
[*********************100%%**********************]  1 of 1 completed
ACN: Sell - Lower Price Target - 174.87
[*********************100%%**********************]  1 of 1 completed
ADBE: Sell - Lower Price Target - 173.54
[*********************100%%**********************]  1 of 1 completed
AIG: Sell - Lower Price Target - 63.89
[*********************100%%**********************]  1 of 1 completed
AMD: Sell - Lower Price Target - 56.91
[*********************100%%**********************]  1 of 1 completed
AMGN: Sell - Lower Price Target - 125.62
[*********************100%%**********************]  1 of 1 completed
AMT: Sell - Lower Price Target - 79.25
[********************

<ipython-input-7-fb2ab7fe1b3b>:52: RuntimeWarning: invalid value encountered in sqrt
  intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)


BKNG: Sell - Lower Price Target - nan
[*********************100%%**********************]  1 of 1 completed
BLK: Buy - Upper Price Target - 1285.12
[*********************100%%**********************]  1 of 1 completed
BMY: Sell - Lower Price Target - 40.20
[*********************100%%**********************]  1 of 1 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



Insufficient data for analysis for BRK.B
[*********************100%%**********************]  1 of 1 completed
C: Buy - Upper Price Target - 90.30
[*********************100%%**********************]  1 of 1 completed
CAT: Sell - Lower Price Target - 165.44
[*********************100%%**********************]  1 of 1 completed
CHTR: Hold - Price Range to Hold - 358.64 to 396.39
[*********************100%%**********************]  1 of 1 completed


...

PM: Sell - Lower Price Target - nan
[*********************100%%**********************]  1 of 1 completed
PYPL: Sell - Lower Price Target - 35.30
[*********************100%%**********************]  1 of 1 completed
QCOM: Sell - Lower Price Target - 66.10
[*********************100%%**********************]  1 of 1 completed
RTX: Sell - Lower Price Target - 63.79
[*********************100%%**********************]  1 of 1 completed


<ipython-input-7-fb2ab7fe1b3b>:52: RuntimeWarning: invalid value encountered in sqrt
  intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)


SBUX: Sell - Lower Price Target - nan
[*********************100%%**********************]  1 of 1 completed
SCHW: Sell - Lower Price Target - 29.82
[*********************100%%**********************]  1 of 1 completed
SO: Sell - Lower Price Target - 55.17
[*********************100%%**********************]  1 of 1 completed
SPG: Sell - Lower Price Target - 86.41
[*********************100%%**********************]  1 of 1 completed


<ipython-input-7-fb2ab7fe1b3b>:52: RuntimeWarning: invalid value encountered in sqrt
  intrinsic_value = np.sqrt(22.5 * earnings_per_share * book_value_per_share)


T: Sell - Lower Price Target - nan
[*********************100%%**********************]  1 of 1 completed
TGT: Sell - Lower Price Target - 89.28
[*********************100%%**********************]  1 of 1 completed
TMO: Sell - Lower Price Target - 442.77
[*********************100%%**********************]  1 of 1 completed
TMUS: Sell - Lower Price Target - 88.49
[*********************100%%**********************]  1 of 1 completed
TSLA: Sell - Lower Price Target - 62.90
[*********************100%%**********************]  1 of 1 completed
TXN: Sell - Lower Price Target - 86.24
[*********************100%%**********************]  1 of 1 completed
UNH: Sell - Lower Price Target - 438.21
[*********************100%%**********************]  1 of 1 completed
UNP: Sell - Lower Price Target - 108.77
[*********************100%%**********************]  1 of 1 completed
UPS: Sell - Lower Price Target - 105.42
[*********************100%%**********************]  1 of 1 completed
USB: Hold - Price Range to